In [13]:
import IPython
from ipywidgets import widgets
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Circle

### Setup other dependencies

In [15]:
def plot_track(t):
    
    tracks_dir = track_directory
    flower_dir = flower_directory

    tracks_plot_dropdown.observe(update_parameters_track, 'value')
    insect_track = pd.read_csv( tracks_dir +str(int(t))+'.csv')
    _speciesnum, _data_point_code, _, _ = get_track_details(t)
    flowers = pd.read_csv(flower_dir+ 'flowers_'+str(_data_point_code)+'.csv')

    fig, ax = plt.subplots(figsize=(19,10))
    _, track_colour  = getSpecies(_speciesnum)
    plt.plot((insect_track['sx_%d'%(int(t))]), ((insect_track['sy_%d'%(int(t))])), color =track_colour)
    plt.scatter((insect_track['x0_%d'%(int(t))]), ((insect_track['yadj_%d'%(int(t))])), color = track_colour, s=3)
    plt.grid()
    
    
    currentAxis = plt.gca()    
    for row in np.arange(0,len(flowers['flower_num'].dropna()),1):
        currentAxis.add_patch(Circle((flowers['x0'][row], flowers['y_adj'][row]), radius = flowers['radius'][row]+20, fill=True, alpha = 0.2))
        currentAxis.add_patch(Circle((flowers['x0'][row], flowers['y_adj'][row]), radius = flowers['radius'][row]+20, fill=False, alpha = 0.8))

    plt.xlim(0,1920)
    plt.ylim(0,1080)
    plt.tight_layout()
    plt.show()

    
def update_parameters_track(t):
    track_label.value = get_text_to_display(tracks_plot_dropdown.value)
    show_image.value = get_insect_image(int(tracks_plot_dropdown.value)).data
    
def get_text_to_display(_track):
    
    _speciesnum, _data_point, _record_date, _sight_time = get_track_details(_track)
    _species, _ = getSpecies(_speciesnum)

    display_text = "<b>Track:</b> "+ str(_track) +"<br>" +"<b>Species:</b> "+ str(_species) + "<br>" + "<b>Location:</b> "+ \
    str(_data_point) +"<br>" +"<b>Record Date:</b> " +str(_record_date) +"<br>" +"<b>Sight Time:</b> "+ str(_sight_time)  
    
    return display_text

def get_insect_image(_track):
    location = image_directory+str(int(_track))+ image_format
    insect_image = IPython.display.Image(location)
    
    return insect_image
 

def get_track_list(directory, track_extension):
    track_list = []
    for track in os.listdir(directory):
        if track.endswith(track_extension):
            try:
                track_list.append(int(track.split('.')[0]))
            except:
                pass
               
    track_list.sort()
    print('Total insect tracks: '+str(len(track_list))+'\n')
    return track_list

def getSpecies(speciesnum):
     
    if speciesnum == 0: 
        _species = 'Honeybee'
        _color = 'tab:blue'
    elif speciesnum == 1: 
        _species = 'Syrphid fly'
        _color = 'tab:orange'
    elif speciesnum == 2: 
        _species = 'Lepidoptera'
        _color = 'tab:green'
    elif speciesnum == 3: 
        _species = 'Vespidae'
        _color = 'tab:red'
    else: 
        _species = 'Other'
        _color = 'tab:grey'
    
    return _species, _color

def get_track_details(t):
    t = str(t)
    speciesnum = int(t[-1])
    data_point = int(t[0:3])
    record_date = int(t[1:3])
    sight_time = t[3:5] + ':' + t[5:7]+ ':' + t[7:9]
    
    return speciesnum, data_point, record_date, sight_time


### Setup iPython widgets

In [16]:
csv_tracks = get_track_list(track_directory, 'csv')

tracks_plot_dropdown = widgets.Dropdown(
    options=csv_tracks,
    value=int(csv_tracks[0]),
    description='Track:',
    disabled=False)


track_label = widgets.HTML(
    value= get_text_to_display(int(csv_tracks[0])),
    disabled=False,
    continuous_update=True)

show_image = widgets.Image(
  value=get_insect_image(csv_tracks[0]).data,
  format='jpg', 
  width=100,
  height=100)


def browse_tracks(track_directory, image_directory, flower_diectory):
    
    

    track_out = widgets.interactive_output(plot_track, {'t':tracks_plot_dropdown})
    
    return widgets.VBox([widgets.HBox([tracks_plot_dropdown,  track_label, show_image]) , track_out])